# 自动求导 Autograd

PyTorch中，所有神经网络的核心是 `autograd` 包。

`autograd` 包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义（define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的。

深度学习的算法本质上是通过反向传播求导数，而 `PyTorch` 的 `autograd` 模块则实现了此功能。在Tensor上的所有操作，`autograd` 都能为它们自动提供微分，避免了手动计算导数的复杂过程。

要想使得Tensor使用autograd功能，只需要设置 `tensor.requries_grad=True`.

## 张量

`torch.Tensor` 是这个包的核心类。如果设置它的属性 `.requires_grad` 为 `True` ，那么它将会追踪对于该张量的所有操作。当完成计算后可以通过调用 `.backward()` ，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到 `.grad` 属性。

要阻止一个张量被跟踪历史，可以调用 `.detach()` 方法将其与计算历史分离，并阻止它未来的计算记录被跟踪。

为了防止跟踪历史记录（和使用内存），可以将代码块包装在 `with torch.no_grad():` 中。在评估模型时特别有用，因为模型可能具有`requires_grad = True` 的可训练的参数，但是我们不需要在此过程中对他们进行梯度计算。

还有一个类对于 `autograd` 的实现非常重要：`Function` 。

`Tensor` 和 `Function` 互相连接生成了一个非循环图，它编码了完整的计算历史。每个张量都有一个 `.grad_fn` 属性，它引用了一个创建了这个`Tensor` 的 `Function`（除非这个张量是用户手动创建的，即这个张量的 `grad_fn` 是 `None`）。

如果需要计算导数，可以在 `Tensor` 上调用 `.backward()` 。如果Tensor是一个标量（即它包含一个元素的数据），则不需要为 `backward()` 指定任何参数，但是如果它有更多的元素，则需要指定一个 `gradient` 参数，它是形状匹配的张量。

In [1]:
import torch

创建一个张量并设置 `requires_grad=True` 用来追踪其计算历史。

In [2]:
# 为tensor设置 requires_grad 标识，代表着需要求导数
# pytorch 会自动调用autograd 记录操作
x = torch.ones(2, 2, requires_grad=True)

# 上一步等价于
# x = t.ones(2,2)
# x.requires_grad = True

print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


对这个张量做一次运算：

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y` 是计算的结果，所以它有 `grad_fn` 属性。

In [4]:
print(y.grad_fn)

对 `y` 进行更多操作

In [5]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad_(...)` 原地改变了现有张量的 `requires_grad` 标志。如果没有指定的话，默认输入的这个标志是 `False`。

In [6]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a)
print(a.requires_grad)

tensor([[ 0.1351, -0.7002],
        [-2.0666, -3.4062]])
False


In [7]:
a.requires_grad_(True)
print(a)
print(a.requires_grad)

tensor([[ 0.1351, -0.7002],
        [-2.0666, -3.4062]], requires_grad=True)
True


In [8]:
b = (a * a).sum()
print(b)

tensor(16.3818, grad_fn=<SumBackward0>)


## 梯度

因为 `out` 是一个标量。所以直接进行反向传播，`out.backward()` 和 `out.backward(torch.tensor(1.))` 等价。

In [9]:
out.backward() # 反向传播,计算梯度

输出导数 `d(out)/dx`

In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以反向传播之前需把梯度清零。

得到都是 `4.5` 的矩阵。调用 `out` 张量 “$o$”，得到：

$$ o = \frac{1}{4}\sum_i z_i $$
$$ z_i = 3(x_i + 2)^2 $$

和
$$ z_i |_{x_i = 1} = 27 $$

因此，
$$ \frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i + 2) $$

所以，
$$ \frac{\partial o}{\partial x_i} |_{x_i=1} = \frac{9}{2} = 4.5 $$

![](https://github.com/pchen12567/picture_store/blob/master/PyTorch/torch_01.png?raw=true)

现在来看一个雅可比向量积的例子:

In [11]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print(y)

tensor([-1231.6356,  -673.9288,  -204.7247], grad_fn=<MulBackward0>)


在这种情况下，`y` 不再是标量。`torch.autograd` 不能直接计算完整的雅可比矩阵，但是如果只想要雅可比向量积，只需将这个向量作为参数传给`backward` ：

In [12]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


为了防止跟踪历史记录（和使用内存），可以将代码块包装在 `with torch.no_grad():` 中。在评估模型时特别有用，因为模型可能具有`requires_grad = True` 的可训练的参数，但是不需要在此过程中对他们进行梯度计算。

也可以通过将代码块包装在 `with torch.no_grad():` 中，来阻止 `autograd` 跟踪设置了 `.requires_grad=True` 的张量的历史记录。

In [13]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
